In [1]:
import sys
from pathlib import Path


In [2]:
import os
from pathlib import Path
from pprint import pprint

In [3]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [4]:
from sqlmodel import Session, select
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint

In [5]:
from api.db.session import engine
from api.events.models import EventModel

In [7]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.user_id.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    print("")
    print(str(query))
    results = session.exec(query).fetchall()
    print("")
    pprint(results)

SELECT events.id, events.time, events.user_id, events.agent_id, events.signal_type, events.emotional_tone, events.drift_score, events.escalate_flag, events.payload, events.relationship_context, events.diagnostic_notes 
FROM events ORDER BY events.user_id ASC
 LIMIT 10

SELECT events.id, events.time, events.user_id, events.agent_id, events.signal_type, events.emotional_tone, events.drift_score, events.escalate_flag, events.payload, events.relationship_context, events.diagnostic_notes 
FROM events ORDER BY events.user_id ASC
 LIMIT :param_1

[EventModel(agent_id='Axis', id=1, emotional_tone=0.72, escalate_flag=0, relationship_context='customer_support', signal_type='relational', time=datetime.datetime(2025, 11, 7, 2, 59, 2, 582611, tzinfo=datetime.timezone.utc), user_id='u_123', drift_score=0.15, payload='{"language": "en-US", "confidence": 0.94, "transcript": "I\'m feeling a bit uncertain about this decision."}', diagnostic_notes='User expressed mild hesitation; tone within normal range

In [23]:
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    signal_types = ["relational", "emotional", "behavioral"]
    query = (
        select(
            bucket.label("bucket"),
            EventModel.signal_type,
            EventModel.agent_id,
            func.avg(EventModel.emotional_tone).label("avg_emotional_tone"),
            func.avg(EventModel.drift_score).label("avg_drift_score"),
            #func.sum(func.case((EventModel.escalate_flag > 0, 1), else_=0)).label("escalate_count"),
            func.count().label("total_count"),
        )
        .where(EventModel.signal_type.in_(signal_types))
        .group_by(
            bucket,
            EventModel.signal_type,
            EventModel.agent_id,
        )
        .order_by(
            bucket,
            EventModel.signal_type,
            EventModel.agent_id,
        )
    )

    compiled_query = query.compile(compile_kwargs={"literal_binds": True})
    print(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)

SELECT time_bucket('1 day'::interval, events.time) AS bucket, events.signal_type, events.agent_id, avg(events.emotional_tone) AS avg_emotional_tone, avg(events.drift_score) AS avg_drift_score, count(*) AS total_count 
FROM events 
WHERE events.signal_type IN ('relational', 'emotional', 'behavioral') GROUP BY time_bucket('1 day'::interval, events.time), events.signal_type, events.agent_id ORDER BY time_bucket('1 day'::interval, events.time), events.signal_type, events.agent_id
[(datetime.datetime(2025, 11, 7, 0, 0, tzinfo=datetime.timezone.utc), 'behavioral', 'Nova', 0.88, 0.39, 1),
 (datetime.datetime(2025, 11, 7, 0, 0, tzinfo=datetime.timezone.utc), 'emotional', 'Axis', 0.65, 0.35, 1),
 (datetime.datetime(2025, 11, 7, 0, 0, tzinfo=datetime.timezone.utc), 'relational', 'Axis', 0.745, 0.135, 2),
 (datetime.datetime(2025, 11, 7, 0, 0, tzinfo=datetime.timezone.utc), 'relational', 'Nova', 0.535, 0.175, 2)]
